In [0]:
#import requests
import pandas as pd
import datetime
import numpy as np
from IPython import display # for updating the cell dynamically

In [0]:
# import os 
# !gunzip /content/drive/My\ Drive/Colab\ Notebooks/Project_4/daily.tar.gz

In [0]:
#import os
# !tar -xf /content/drive/My\ Drive/Colab\ Notebooks/Project_4/sfc.tar --directory /content/drive/My\ Drive/Colab\ Notebooks/Project_4/GFSSfc/

In [0]:
## Read in CSV files

#KCMI_daily_data = pd.read_csv('https://raw.githubusercontent.com/jdthayer/597_Project4_GroupB/master/raw_data/KCMI_dailyNOFOOTER.csv', header=8)

#KCMI_daily_data

In [0]:
import pandas as pd
import glob

path = '/content/drive/My Drive/Colab Notebooks/Project_4/GFSSfc/bufkit/' # CHANGE TO YOUR FILE PATH FOR THE UNTARRED FILES
all_files = glob.glob(path + "*.csv")
all_files.sort()

GFS_array = []

for i in np.arange(0,1):

  filename = all_files[i].split(".")
  filename1 = filename[0].split('/')[-1]
  datestring = filename1[0:4] + '-' + filename1[4:6] + '-' + filename1[6:8] + ' ' + filename1[8:10] + 'Z'

  if int(filename1[0:4]) < 2020: ## Only using files with an initialization date before 2020-01-01

    if i == 0: ## Only appending column names for first file
      GFS_file_df = pd.read_csv(all_files[i], header=0)
      GFS_file_df = GFS_file_df.T # Transposing dataframe
      GFS_file_df = GFS_file_df.rename(index={'Unnamed: 0':'Date'}) # Changing first value within index column to 'Date'
      GFS_file_df = GFS_file_df.drop(GFS_file_df.columns[[1, 3, 5, 7, 8]], axis=1) # Retaining only Temp, Wind Speed, Precip columns
      GFS_file_df = GFS_file_df[[0,2,6,9,4]] # Re-arranging columns to the same order as the KCMI data
      GFS_array.append(GFS_file_df.iloc[:, 0:3]) # Only appending 06Z-06Z forecasts
    else: ## Only appending 06Z-06Z data after the first file
      GFS_file_df = pd.read_csv(all_files[i], header=0)
      GFS_file_df = GFS_file_df.T # Transposing dataframe
      GFS_file_df = GFS_file_df.drop(GFS_file_df.columns[[1, 3, 5, 7, 8]], axis=1) # Retaining only Temp, Wind Speed, Precip columns
      GFS_file_df = GFS_file_df[[0,2,6,9,4]] # Re-arranging columns to the same order as the KCMI data
      GFS_array.append(GFS_file_df.iloc[1:, 0:3]) # Only appending 06Z-06Z forecasts

GFS_Sfc_42hrFcst = pd.concat(GFS_array, axis=0)
#GFS_Sfc_42hrFcst.to_csv('/content/drive/My Drive/Colab Notebooks/Project_4/GFSSfc/GFS_Sfc_42hrFcst_2010thru2019.csv')

In [0]:
## GFS Sfc data calculations from 06Z-06Z

# Creating new dataframe
index = pd.date_range(start="2010-01-01 12:00:00",end="2019-12-31 12:00:00")
columns = ['Avg DWPDEP(C)', 'Min DWPDEP(C)', 'Max DWPDEP(C)',
           'Avg LCLD(%)', 'Min LCLD(%)', 'Max LCLD(%)', 
           'Avg MCLD(%)', 'Min MCLD(%)', 'Max MCLD(%)',
           'Avg HCLD(%)', 'Min HCLD(%)', 'Max HCLD(%)',
           'Avg PRES(hPa)', 'Min PRES(hPa)', 'Max PRES(hPa)']
GFS_Sfc_df = pd.DataFrame(index=index, columns=columns)
GFS_Sfc_df.columns.name = 'Date'

import pandas as pd
import glob

path = '/content/drive/My Drive/Colab Notebooks/Project_4/GFSSfc/bufkit/' # CHANGE TO YOUR FILE PATH FOR THE UNTARRED FILES
all_files = glob.glob(path + "*.csv")
all_files.sort()

for i in np.arange(0,len(all_files)):

  filename = all_files[i].split(".")
  filename1 = filename[0].split('/')[-1]
  #datestring = filename1[0:4] + '-' + filename1[4:6] + '-' + filename1[6:8] + ' ' + filename1[8:10] + 'Z'

  if int(filename1[0:4]) < 2020: ## Only using files with an initialization date before 2020-01-01

    GFS_file_df = pd.read_csv(all_files[i], header=0)
    GFS_file_df = GFS_file_df.T # Transposing dataframe

    # Variable calculations for 06Z-06Z, excluding -9999 values for cloud cover
    DewptDep = GFS_file_df[6].values[1:] - GFS_file_df[0].values[1:]
    DewptDepAVG = DewptDep.mean()
    DewptDepMIN = DewptDep.min()
    DewptDepMAX = DewptDep.max()
    GFS_Sfc_df['Avg DWPDEP(C)'][i] = DewptDepAVG
    GFS_Sfc_df['Min DWPDEP(C)'][i] = DewptDepMIN
    GFS_Sfc_df['Max DWPDEP(C)'][i] = DewptDepMAX

    GFS_file_df[2].values[1:][np.where(GFS_file_df[2].values[1:] < 0)] = np.nan
    LCLDAVG = GFS_file_df[2].values[1:][~pd.isnull(GFS_file_df[2].values[1:])].mean() ## nanmean for arrays of dtype=object
    LCLDMIN = GFS_file_df[2].values[1:][~pd.isnull(GFS_file_df[2].values[1:])].min() ## nanmin for arrays of dtype=object
    LCLDMAX = GFS_file_df[2].values[1:][~pd.isnull(GFS_file_df[2].values[1:])].max() ## nanmax for arrays of dtype=object
    GFS_Sfc_df['Avg LCLD(%)'][i] = LCLDAVG
    GFS_Sfc_df['Min LCLD(%)'][i] = LCLDMIN
    GFS_Sfc_df['Max LCLD(%)'][i] = LCLDMAX

    GFS_file_df[3].values[1:][np.where(GFS_file_df[3].values[1:] < 0)] = np.nan
    MCLDAVG = GFS_file_df[3].values[1:][~pd.isnull(GFS_file_df[3].values[1:])].mean() ## nanmean for arrays of dtype=object
    MCLDMIN = GFS_file_df[3].values[1:][~pd.isnull(GFS_file_df[3].values[1:])].min() ## nanmin for arrays of dtype=object
    MCLDMAX = GFS_file_df[3].values[1:][~pd.isnull(GFS_file_df[3].values[1:])].max() ## nanmax for arrays of dtype=object
    GFS_Sfc_df['Avg MCLD(%)'][i] = MCLDAVG
    GFS_Sfc_df['Min MCLD(%)'][i] = MCLDMIN
    GFS_Sfc_df['Max MCLD(%)'][i] = MCLDMAX

    GFS_file_df[1].values[1:][np.where(GFS_file_df[1].values[1:] < 0)] = np.nan
    HCLDAVG = GFS_file_df[1].values[1:][~pd.isnull(GFS_file_df[1].values[1:])].mean() ## nanmean for arrays of dtype=object
    HCLDMIN = GFS_file_df[1].values[1:][~pd.isnull(GFS_file_df[1].values[1:])].min() ## nanmean for arrays of dtype=object
    HCLDMAX = GFS_file_df[1].values[1:][~pd.isnull(GFS_file_df[1].values[1:])].max() ## nanmean for arrays of dtype=object
    GFS_Sfc_df['Avg HCLD(%)'][i] = HCLDAVG
    GFS_Sfc_df['Min HCLD(%)'][i] = HCLDMIN
    GFS_Sfc_df['Max HCLD(%)'][i] = HCLDMAX

    PRESAVG = GFS_file_df[5].values[1:].mean()
    PRESMIN = GFS_file_df[5].values[1:].min()
    PRESMAX = GFS_file_df[5].values[1:].max()
    GFS_Sfc_df['Avg PRES(hPa)'][i] = PRESAVG
    GFS_Sfc_df['Min PRES(hPa)'][i] = PRESMIN
    GFS_Sfc_df['Max PRES(hPa)'][i] = PRESMAX

#GFS_Sfc_df.to_csv('/content/drive/My Drive/Colab Notebooks/Project_4/GFSSfc/GFS_Sfc_06Zto06ZAvgMinMax_AdditionalVars_2010thru2019.csv')

In [0]:
## GFS Profile data calculations from 06Z-06Z

# Creating new dataframe
index = pd.date_range(start="2010-01-01 12:00:00",end="2019-12-31 12:00:00")
columns = ['Avg 850hPa DWPDEP(C)', 'Min 850hPa DWPDEP(C)', 'Max 850hPa DWPDEP(C)',
           'Avg 925hPa DWPDEP(C)', 'Min 925hPa DWPDEP(C)', 'Max 925hPa DWPDEP(C)',
           'Avg 250hPa HGT(m)', 'Min 250hPa HGT(m)', 'Max 250hPa HGT(m)',
           'Avg 500hPa HGT(m)', 'Min 500hPa HGT(m)', 'Max 500hPa HGT(m)',
           'Min 850hPa TMPC', 'Max 850hPa TMPC',
           'Min 925hPa TMPC', 'Max 925hPa TMPC', 
           'Min 250hPa WSPD(m/s)', 'Max 250hPa WSPD(m/s)',
           'Min 500hPa WSPD(m/s)', 'Max 500hPa WSPD(m/s)',
           'Min 850hPa WSPD(m/s)', 'Max 850hPa WSPD(m/s)',
           'Min 925hPa WSPD(m/s)', 'Max 925hPa WSPD(m/s)']
GFS_Prof_df = pd.DataFrame(index=index, columns=columns)
GFS_Prof_df.columns.name = 'Date'

import pandas as pd
import glob

path = '/content/drive/My Drive/Colab Notebooks/Project_4/GFSProfile/bufkit/' # CHANGE TO YOUR FILE PATH FOR THE UNTARRED FILES
all_files = glob.glob(path + "*.csv")
all_files.sort()

for i in np.arange(0,len(all_files)):

  filename = all_files[i].split(".")
  filename1 = filename[0].split('/')[-1]

  if int(filename1[0:4]) < 2020: ## Only using files with an initialization date before 2020-01-01

    GFS_file_df = pd.read_csv(all_files[i], header=0)

    # Variable calculations for 06Z-06Z
    DWPDEP_850 = []
    DWPDEP_925 = []
    for j in np.arange(0,9): ## cycling through 06Z-06Z every 3 hours
      TMPC = GFS_file_df['TMPC'][j]
      DWPC = GFS_file_df['DWPC'][j]
      TMPC_850 = float(TMPC[1:-2].split(',')[1])
      DWPC_850 = float(DWPC[1:-2].split(',')[1])
      TMPC_925 = float(TMPC[1:-2].split(',')[0])
      DWPC_925 = float(DWPC[1:-2].split(',')[0])
      DWPDEP_850.append(TMPC_850 - DWPC_850)
      DWPDEP_925.append(TMPC_925 - DWPC_925)
    DWPDEP_850_AVG = np.array(DWPDEP_850).mean()
    DWPDEP_850_MIN = np.array(DWPDEP_850).min()
    DWPDEP_850_MAX = np.array(DWPDEP_850).max()
    DWPDEP_925_AVG = np.array(DWPDEP_925).mean()
    DWPDEP_925_MIN = np.array(DWPDEP_925).min()
    DWPDEP_925_MAX = np.array(DWPDEP_925).max()
    GFS_Prof_df['Avg 850hPa DWPDEP(C)'][i] = DWPDEP_850_AVG
    GFS_Prof_df['Min 850hPa DWPDEP(C)'][i] = DWPDEP_850_MIN
    GFS_Prof_df['Max 850hPa DWPDEP(C)'][i] = DWPDEP_850_MAX
    GFS_Prof_df['Avg 925hPa DWPDEP(C)'][i] = DWPDEP_925_AVG
    GFS_Prof_df['Min 925hPa DWPDEP(C)'][i] = DWPDEP_925_MIN
    GFS_Prof_df['Max 925hPa DWPDEP(C)'][i] = DWPDEP_925_MAX

    HGT_250 = []
    HGT_500 = []
    for j in np.arange(0,9): ## cycling through 06Z-06Z every 3 hours
      HGT = GFS_file_df['HGHT'][j]
      HGT_250.append(float(HGT[1:-2].split(',')[4]))
      HGT_500.append(float(HGT[1:-2].split(',')[3]))
    HGT_250_AVG = np.array(HGT_250).mean()
    HGT_250_MIN = np.array(HGT_250).min()
    HGT_250_MAX = np.array(HGT_250).max()
    HGT_500_AVG = np.array(HGT_500).mean()
    HGT_500_MIN = np.array(HGT_500).min()
    HGT_500_MAX = np.array(HGT_500).max()
    GFS_Prof_df['Avg 250hPa HGT(m)'][i] = HGT_250_AVG
    GFS_Prof_df['Min 250hPa HGT(m)'][i] = HGT_250_MIN
    GFS_Prof_df['Max 250hPa HGT(m)'][i] = HGT_250_MAX
    GFS_Prof_df['Avg 500hPa HGT(m)'][i] = HGT_500_AVG
    GFS_Prof_df['Min 500hPa HGT(m)'][i] = HGT_500_MIN
    GFS_Prof_df['Max 500hPa HGT(m)'][i] = HGT_500_MAX

    TMPC_850 = []
    TMPC_925 = []
    for j in np.arange(0,9): ## cycling through 06Z-06Z every 3 hours
      TMPC = GFS_file_df['TMPC'][j]
      TMPC_850.append(float(TMPC[1:-2].split(',')[1]))
      TMPC_925.append(float(TMPC[1:-2].split(',')[0]))
    TMPC_850_MIN = np.array(TMPC_850).min()
    TMPC_850_MAX = np.array(TMPC_850).max()
    TMPC_925_MIN = np.array(TMPC_925).min()
    TMPC_925_MAX = np.array(TMPC_925).max()
    GFS_Prof_df['Min 850hPa TMPC'][i] = TMPC_850_MIN
    GFS_Prof_df['Max 850hPa TMPC'][i] = TMPC_850_MAX
    GFS_Prof_df['Min 925hPa TMPC'][i] = TMPC_925_MIN
    GFS_Prof_df['Max 925hPa TMPC'][i] = TMPC_925_MAX

    WSPD_250 = []
    WSPD_500 = []
    WSPD_850 = []
    WSPD_925 = []
    for j in np.arange(0,9): ## cycling through 06Z-06Z every 3 hours
      UWND = GFS_file_df['UWND'][j]
      VWND = GFS_file_df['VWND'][j]
      UWND_250 = float(UWND[1:-2].split(',')[4])
      VWND_250 = float(VWND[1:-2].split(',')[4])
      UWND_500 = float(UWND[1:-2].split(',')[3])
      VWND_500 = float(VWND[1:-2].split(',')[3])
      UWND_850 = float(UWND[1:-2].split(',')[1])
      VWND_850 = float(VWND[1:-2].split(',')[1])
      UWND_925 = float(UWND[1:-2].split(',')[0])
      VWND_925 = float(VWND[1:-2].split(',')[0])
      WSPD_250.append(np.sqrt(UWND_250**2 + VWND_250**2))
      WSPD_500.append(np.sqrt(UWND_500**2 + VWND_500**2))
      WSPD_850.append(np.sqrt(UWND_850**2 + VWND_850**2))
      WSPD_925.append(np.sqrt(UWND_925**2 + VWND_925**2))
    WSPD_250_MIN = np.array(WSPD_250).min()
    WSPD_250_MAX = np.array(WSPD_250).max()
    WSPD_500_MIN = np.array(WSPD_500).min()
    WSPD_500_MAX = np.array(WSPD_500).max()
    WSPD_850_MIN = np.array(WSPD_850).min()
    WSPD_850_MAX = np.array(WSPD_850).max()
    WSPD_925_MIN = np.array(WSPD_925).min()
    WSPD_925_MAX = np.array(WSPD_925).max()
    GFS_Prof_df['Min 250hPa WSPD(m/s)'][i] = WSPD_250_MIN
    GFS_Prof_df['Max 250hPa WSPD(m/s)'][i] = WSPD_250_MAX
    GFS_Prof_df['Min 500hPa WSPD(m/s)'][i] = WSPD_500_MIN
    GFS_Prof_df['Max 500hPa WSPD(m/s)'][i] = WSPD_500_MAX
    GFS_Prof_df['Min 850hPa WSPD(m/s)'][i] = WSPD_850_MIN
    GFS_Prof_df['Max 850hPa WSPD(m/s)'][i] = WSPD_850_MAX
    GFS_Prof_df['Min 925hPa WSPD(m/s)'][i] = WSPD_925_MIN
    GFS_Prof_df['Max 925hPa WSPD(m/s)'][i] = WSPD_925_MAX

#GFS_Prof_df.to_csv('/content/drive/My Drive/Colab Notebooks/Project_4/GFSProfile/GFS_Prof_06Zto06ZAvgMinMax_2010thru2019.csv')

In [0]:
# GFS_file_df = pd.read_csv(all_files[1], header=0)
# GFS_file_df = GFS_file_df.T # Transposing dataframe
# GFS_file_df[1].values[1:][np.where(GFS_file_df[1].values[1:] < 0)] = np.nan
# A = GFS_file_df[1].values[1:]
# GFS_file_df[1].values[1:][~pd.isnull(GFS_file_df[1].values[1:])].mean()

7.375

In [0]:
# import pandas as pd
# import glob

# path = '/content/drive/My Drive/Colab Notebooks/Project_4/GFSDaily/bufkit/' # CHANGE TO YOUR FILE PATH FOR THE UNTARRED FILES
# all_files = glob.glob(path + "*.csv")
# all_files.sort()

# GFS_array = []

# for i in np.arange(0,len(all_files)):

#   filename = all_files[i].split(".")
#   filename1 = filename[0].split('/')[-1]

#   if int(filename1[0:4]) < 2020: ## Only using files with an initialization date before 2020-01-01

#     if i == 0: ## Only appending column names for first file
#       GFS_file_df = pd.read_csv(all_files[i], header=0)
#       GFS_file_df = GFS_file_df.rename(columns={'Unnamed: 0':'Date'}) # Changing first value within index column to 'Date'
#       GFS_array.append(GFS_file_df) # Only appending data
#     else: ## Only appending data after the first file
#       GFS_file_df = pd.read_csv(all_files[i], header=0)
#       GFS_file_df = GFS_file_df.rename(columns={'Unnamed: 0':'Date'}) # Changing first value within index column to 'Date'
#       GFS_array.append(GFS_file_df)

# GFS_Daily_PredictVars = pd.concat(GFS_array, axis=0)
#GFS_Daily_PredictVars.to_csv('/content/drive/My Drive/Colab Notebooks/Project_4/GFSDaily/GFS_Daily_PredictVars_2010thru2019.csv')

In [0]:
GFS_file_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Project_4/GFSProfile/bufkit/2010010112.gfs_kcmi.buf_prof.csv', header=0)
# GFS_file_df = GFS_file_df.T # transpose dataframe
# GFS_file_df = GFS_file_df.rename(columns={'Unnamed: 0':'Date'})
# GFS_file_df = GFS_file_df.drop(GFS_file_df.columns[[0, 1, 2, 3, 5, 7, 8]], axis=1) # retaining only Temp, Wind Speed, Precip columns
# GFS_file_df = GFS_file_df[[6,9,4]] # re-arranging columns to the same order as the KCMI data
# GFS_file_df.iloc[:, 0:3] # 42-hr forecast
A = GFS_file_df['TMPC'][:]
GFS_file_df
#float(A[1:-2].split(',')[0])

,Unnamed: 0,DWPC,HGHT,PRES,TMPC,UWND,VWND
0,2010-01-02 06:00:00,"[-17.885416666666668, -22.15150234741784, -38....","[822.2693055555555, 1460.1033333333335, 2904.4...","[925.0, 850.0, 700.0, 500.0, 250.0, 100.0]","[-16.249583333333334, -16.52291079812207, -21....","[10.566530573750587, 4.214706046596407, 17.668...","[-21.60349159268254, -18.30022411700173, -29.5..."
1,2010-01-02 09:00:00,"[-19.453124999999996, -24.9388679245283, -42.0...","[822.0543749999998, 1459.7785849056602, 2907.4...","[925.0, 850.0, 700.0, 500.0, 250.0, 100.0]","[-17.247500000000002, -16.19490566037736, -20....","[13.681992680084207, 9.948942540840713, 19.953...","[-21.05132415236028, -17.798267800386434, -29...."
2,2010-01-02 12:00:00,"[-24.917708333333334, -29.536603773584904, -42...","[820.327638888889, 1459.582075471698, 2915.667...","[925.0, 850.0, 700.0, 500.0, 250.0, 100.0]","[-15.991249999999997, -15.280283018867925, -19...","[12.509088069631126, 14.999567192631927, 23.33...","[-25.372665670766228, -22.851012927878898, -32..."
3,2010-01-02 15:00:00,"[-29.030482758620682, -37.651971830985914, -43...","[826.2597241379308, 1463.5210798122066, 2927.6...","[925.0, 850.0, 700.0, 500.0, 250.0, 100.0]","[-18.595172413793104, -14.76, -16.487358490566...","[11.080763032604898, 13.737749044170894, 22.27...","[-20.65371472040801, -25.032875351390114, -38...."
4,2010-01-02 18:00:00,"[-24.84875862068967, -32.677452830188685, -41....","[823.4282758620693, 1458.4197169811318, 2927.1...","[925.0, 850.0, 700.0, 500.0, 250.0, 100.0]","[-19.58413793103448, -15.190188679245283, -15....","[11.173738737104433, 11.725204010079779, 25.07...","[-14.419104627678692, -22.850212323925128, -37..."
5,2010-01-02 21:00:00,"[-23.723541666666648, -31.231603773584904, -41...","[814.5235416666663, 1451.7584905660376, 2924.7...","[925.0, 850.0, 700.0, 500.0, 250.0, 100.0]","[-18.52666666666667, -14.439245283018868, -14....","[9.280847437628964, 13.398593774093092, 28.752...","[-16.548989694297227, -22.417731320168485, -37..."
6,2010-01-03 00:00:00,"[-27.292965517241385, -32.07079812206573, -41....","[818.8129655172417, 1456.3933333333334, 2930.5...","[925.0, 850.0, 700.0, 500.0, 250.0, 100.0]","[-17.499310344827585, -14.52056338028169, -14....","[3.511034750622741, 11.005251667788372, 30.469...","[-18.2430697535647, -23.400538147416142, -38.7..."
7,2010-01-03 03:00:00,"[-27.28510344827586, -33.31539906103286, -43.7...","[820.7152413793101, 1458.8048356807508, 2934.9...","[925.0, 850.0, 700.0, 500.0, 250.0, 100.0]","[-16.711724137931036, -14.65342723004695, -13....","[1.184391613594784, 6.8638483403032735, 24.840...","[-14.134740826662464, -19.89592389230682, -38...."
8,2010-01-03 06:00:00,"[-25.6275, -33.05830188679246, -45.49559748427...","[820.549166666667, 1458.3049056603775, 2932.43...","[925.0, 850.0, 700.0, 500.0, 250.0, 100.0]","[-16.143333333333334, -15.216603773584906, -13...","[-1.9754764134434684, 3.0071145961088406, 20.2...","[-8.494140743342218, -13.48674534777446, -32.1..."
